In [1]:
import os
import json
import math
from datetime import datetime, timedelta
from typing import Optional, Union

import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point


In [2]:
root_path = os.path.abspath('..')
bounding_file = os.path.join(root_path, 'data/boundary.geojson')

In [3]:
bound_gdf = gpd.read_file(bounding_file, driver='GeoJSON')

In [30]:

def get_stations_from_bbox():
    """Return a list of stations IDs found within a bounding box.
    Returns:
        GeoDataFrame[str]: A geodataframe with all stations' coordinates, id, and name
    """
    
    data_url = "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations.json"
    response = requests.get(data_url)
    json_dict = response.json()
    
    stations = []
    for station in json_dict["stations"]:
        station_info = []
        station_info.append(station['name'])
        station_info.append(station['id'])
        station_info.append(station['lat'])
        station_info.append(station['lng'])
        
        stations.append(station_info)
    
    df = pd.DataFrame(stations, columns=['name', 'id', 'lat', 'lng'])
    geometry = [Point(xy) for xy in zip(df.lng, df.lat)]
    points_gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

    return points_gdf

In [31]:
points = get_stations_from_bbox()

In [52]:
sub_stations = points[points.within(bound_gdf.loc[0, 'geometry'])]
sub_stations.to_csv('stations.csv', index=False)